In [2]:
import subprocess
import os
from glob import glob
import pandas as pd
import nibabel as nib
import numpy as np

In [ ]:
# 14 SVBP => 1,761 (95%)

In [ ]:
# 9 MAP6 => 1.833 (95%)

In [22]:
folders = [
    "ith_lambda1_3",
    "lambda1_1.0",    
]
for i, folder in enumerate(folders):
    jacs = sorted(glob(os.path.join("output", "GIN", folder, "Log10Jacobian_Deformable", "*.nii.gz")))
    for jac in jacs:
        outdir = os.path.join("output", "GIN", folder, "Log10Jac_Gaussian")
        try:
            os.mkdir(outdir)
        except:
            pass
        id = jac.split('/')[-1].split('_')[1]
        output = os.path.join(outdir, "Log10Jac_Gaussian_" + id + ".nii.gz")
        print("fslmaths " + jac + " -s 2 " + output)
    print('-'*20)

fslmaths output/GIN/ith_lambda1_3/Log10Jacobian_Deformable/Log10JacobianDeformable_653_N4_o_128_ras_norm_id_affine_warp.nii.gz -s 2 output/GIN/ith_lambda1_3/Log10Jac_Gaussian/Log10Jac_Gaussian_653.nii.gz
fslmaths output/GIN/ith_lambda1_3/Log10Jacobian_Deformable/Log10JacobianDeformable_655_N4_o_128_ras_norm_id_affine_warp.nii.gz -s 2 output/GIN/ith_lambda1_3/Log10Jac_Gaussian/Log10Jac_Gaussian_655.nii.gz
fslmaths output/GIN/ith_lambda1_3/Log10Jacobian_Deformable/Log10JacobianDeformable_656_N4_o_128_ras_norm_id_affine_warp.nii.gz -s 2 output/GIN/ith_lambda1_3/Log10Jac_Gaussian/Log10Jac_Gaussian_656.nii.gz
fslmaths output/GIN/ith_lambda1_3/Log10Jacobian_Deformable/Log10JacobianDeformable_660_N4_o_128_ras_norm_id_affine_warp.nii.gz -s 2 output/GIN/ith_lambda1_3/Log10Jac_Gaussian/Log10Jac_Gaussian_660.nii.gz
fslmaths output/GIN/ith_lambda1_3/Log10Jacobian_Deformable/Log10JacobianDeformable_661_N4_o_128_ras_norm_id_affine_warp.nii.gz -s 2 output/GIN/ith_lambda1_3/Log10Jac_Gaussian/Log10Jac_

In [29]:
folders = [
    "ith_lambda1_3",
    "lambda1_1.0",    
]
names = [
    "Pair_noaug",
    "OP"
]
types = [
    "WT",
    "KO",
]
subs = [
    "SVBP",
    "MAP6"
]

gin_csv = os.path.join("dataset3", "GIN", "gin.csv")
gin = pd.read_csv(gin_csv)

for i, folder in enumerate(folders):
    for sub in subs:
        std_jac = os.path.join("output", "GIN", folder, "std_jac.nii.gz")
        std_jac_nib = nib.load(std_jac)
        jacs = sorted(glob(os.path.join("output", "GIN", folder, "Log10Jac_Gaussian", "*.nii.gz")))
        for type in types:
            count = 0
            mean = np.zeros(std_jac_nib.shape)
            for jac in jacs:
                name_id = jac.split('/')[-1].split('_')[2].split('.')[0]
                type_id = gin[gin['name'] == name_id]['type'].iloc[0]
                sub_id = gin[gin['name'] == name_id]['study'].iloc[0]
                if type_id == type and sub_id == sub:
                    count = count + 1
                    mean = mean + nib.load(jac).get_fdata()     
            print(count)
            mean = mean / count
            out_name = "mean_jacgaussian_" + str(type) + "_" + str(sub) + ".nii.gz"
            target = os.path.join("output", "GIN", folder, out_name)
            image = nib.Nifti1Image(mean, std_jac_nib.affine, std_jac_nib.header)
            nib.save(image, target)
            print(target)
            print('-'*10)
            
for i, folder in enumerate(folders):
    for sub in subs:
        std_jac = os.path.join("output", "GIN", folder, "std_jac.nii.gz")
        std_jac_nib = nib.load(std_jac)
        jacs = sorted(glob(os.path.join("output", "GIN", folder, "Log10Jac_Gaussian", "*.nii.gz")))
        for type in types:
            count = 0
            mean_jac = nib.load(os.path.join("output", "GIN", folder, "mean_jacgaussian_" + str(type) + "_" + str(sub) + ".nii.gz")).get_fdata()
            std = np.zeros(std_jac_nib.shape)
            for jac in jacs:
                name_id = jac.split('/')[-1].split('_')[2].split('.')[0]
                type_id = gin[gin['name'] == name_id]['type'].iloc[0]                
                sub_id = gin[gin['name'] == name_id]['study'].iloc[0]
                if type_id == type and sub_id == sub:
                    count = count + 1
                    std = std + (nib.load(jac).get_fdata())**2   
            print(count)
            std = (std / count - mean_jac**2)**(1/2)
            image = nib.Nifti1Image(std, std_jac_nib.affine, std_jac_nib.header)
            out_name = "std_jacgaussian_" + str(type) + "_" + str(sub) + ".nii.gz"
            target = os.path.join("output", "GIN", folder, out_name)
            nib.save(image, target)
            print(target)
            print('-'*10)
            
for i, folder in enumerate(folders):
    std_jac = os.path.join("output", "GIN", folder, "std_jac.nii.gz")
    std_jac_nib = nib.load(std_jac)
    jacs = sorted(glob(os.path.join("output", "GIN", folder, "Log10Jac_Gaussian", "*.nii.gz")))
    count = 0
    mean = np.zeros(std_jac_nib.shape)
    for jac in jacs:
        name_id = jac.split('/')[-1].split('_')[2].split('.')[0]
        type_id = gin[gin['name'] == name_id]['type'].iloc[0]
        sub_id = gin[gin['name'] == name_id]['study'].iloc[0]
        count = count + 1
        mean = mean + nib.load(jac).get_fdata()     
    print(count)
    mean = mean / count
    out_name = "mean_jacgaussian.nii.gz"
    target = os.path.join("output", "GIN", folder, out_name)
    image = nib.Nifti1Image(mean, std_jac_nib.affine, std_jac_nib.header)
    nib.save(image, target)
    print(target)
    print('-'*10)
            
for i, folder in enumerate(folders):
    std_jac = os.path.join("output", "GIN", folder, "std_jac.nii.gz")
    std_jac_nib = nib.load(std_jac)
    jacs = sorted(glob(os.path.join("output", "GIN", folder, "Log10Jac_Gaussian", "*.nii.gz")))
    count = 0
    mean_jac = nib.load(os.path.join("output", "GIN", folder, "mean_jacgaussian.nii.gz")).get_fdata()
    std = np.zeros(std_jac_nib.shape)
    for jac in jacs:
        name_id = jac.split('/')[-1].split('_')[2].split('.')[0]
        type_id = gin[gin['name'] == name_id]['type'].iloc[0] 
        count = count + 1
        std = std + (nib.load(jac).get_fdata())**2 
    print(count)
    std = (std / count - mean_jac**2)**(1/2)
    image = nib.Nifti1Image(std, std_jac_nib.affine, std_jac_nib.header)
    out_name = "std_jacgaussian.nii.gz"
    target = os.path.join("output", "GIN", folder, out_name)
    nib.save(image, target)
    print(target)
    print('-'*10)
            
for i, folder in enumerate(folders):
    for sub in subs:
        std_jac_global = nib.load(os.path.join("output", "GIN", folder, "std_jacgaussian.nii.gz"))
        affine = std_jac_global.affine
        header = std_jac_global.header
        std_jac_global = std_jac_global.get_fdata()        
        mean_jac_wt = nib.load(os.path.join("output", "GIN", folder, "mean_jacgaussian_WT_" + sub + ".nii.gz")).get_fdata()
        mean_jac_ko = nib.load(os.path.join("output", "GIN", folder, "mean_jacgaussian_KO_" + sub + ".nii.gz")).get_fdata()        
        std_jac_wt = nib.load(os.path.join("output", "GIN", folder, "std_jacgaussian_WT_" + sub + ".nii.gz")).get_fdata()
        std_jac_ko = nib.load(os.path.join("output", "GIN", folder, "std_jacgaussian_KO_" + sub + ".nii.gz")).get_fdata()
        
        diff = (mean_jac_ko - mean_jac_wt) / std_jac_global
        target = os.path.join("output", "GIN", folder, "meandiffstd_jacgaussian_" + sub + "_" + names[i] + ".nii.gz")
        image = nib.Nifti1Image(diff, affine, header)
        nib.save(image, target)
        
        if sub == 'MAP6':
            n_ko = 5
            n_wt = 6
        else:
            n_ko = 8
            n_wt = 8    
        
        sp = (((n_wt-1)*std_jac_wt**2 + (n_ko-1)*std_jac_ko**2)/(n_wt+n_ko-2))**(1/2)
        denom = sp * (1/n_wt + 1/n_ko)**(1/2)
        t_test = (mean_jac_ko - mean_jac_wt) / denom
        image = nib.Nifti1Image(t_test, affine, header)        
        target = os.path.join("output", "GIN", folder, "ttest_jacgaussian_" + sub + "_" + names[i] + ".nii.gz")
        print(target)
        print('-'*10)
        nib.save(image, target)       

8
output/GIN/ith_lambda1_3/mean_jacgaussian_WT_SVBP.nii.gz
----------
8
output/GIN/ith_lambda1_3/mean_jacgaussian_KO_SVBP.nii.gz
----------
6
output/GIN/ith_lambda1_3/mean_jacgaussian_WT_MAP6.nii.gz
----------
5
output/GIN/ith_lambda1_3/mean_jacgaussian_KO_MAP6.nii.gz
----------
8
output/GIN/lambda1_1.0/mean_jacgaussian_WT_SVBP.nii.gz
----------
8
output/GIN/lambda1_1.0/mean_jacgaussian_KO_SVBP.nii.gz
----------
6
output/GIN/lambda1_1.0/mean_jacgaussian_WT_MAP6.nii.gz
----------
5
output/GIN/lambda1_1.0/mean_jacgaussian_KO_MAP6.nii.gz
----------
8
output/GIN/ith_lambda1_3/std_jacgaussian_WT_SVBP.nii.gz
----------
8
output/GIN/ith_lambda1_3/std_jacgaussian_KO_SVBP.nii.gz
----------
6
output/GIN/ith_lambda1_3/std_jacgaussian_WT_MAP6.nii.gz
----------
5
output/GIN/ith_lambda1_3/std_jacgaussian_KO_MAP6.nii.gz
----------
8
output/GIN/lambda1_1.0/std_jacgaussian_WT_SVBP.nii.gz
----------
8
output/GIN/lambda1_1.0/std_jacgaussian_KO_SVBP.nii.gz
----------
6
output/GIN/lambda1_1.0/std_jacgaussi

In [21]:
studies = ["SyN",
           "SyN-Mean",
           "SyN-Mask",
           "DL-IRIS",
           "DL-GIN-3fold",
           "OP",
           "Pair",
           "Pair-aug",
           "Pair-zero",
           "Pair-null",
          ]

folders = [
    'ANTS',
    'ANTS-Mean',
    'ANTS-Mask',
    'dl-iris',
    'Gin_kfold',
    'lambda1_1.0',
    'ith_lambda1_3',
    'ith_best_aug_nosym',
    'pairwise_zero_nosym_fix',
    'ith_null',
]

types = [
    "WT",
    "KO",
]
subs = [
    "SVBP",
    "MAP6"
]

gin_csv = os.path.join("dataset3", "GIN", "gin.csv")
kfold_csv = "dataset3/GI3N/kfold_mapping.csv"
kfold = pd.read_csv(kfold_csv)
gin = pd.read_csv(gin_csv)
std_jac = os.path.join("output", "GIN", "ith_lambda1_3", "std_jac.nii.gz")
std_jac_nib = nib.load(std_jac)
affine = std_jac_nib.affine
header = std_jac_nib.header
for i, folder in enumerate(folders):
    for sub in subs:        
        jacs = sorted(glob(os.path.join("output", "GIN", folder, "Log10Jacobian_Deformable", "*.nii.gz")))
        for type in types:
            count = 0
            mean = np.zeros(std_jac_nib.shape)
            for jac in jacs:
                name_id = jac.split('/')[-1].split('_')[1]
                if 'kfold' in folder:
                    name_kfold = jac.split('/')[-1].split('_')[2] + '_' + jac.split('/')[-1].split('_')[3]
                    name_id = kfold[kfold['fold_name'] == name_kfold]['name'].iloc[0]
                type_id = gin[gin['name'] == name_id]['type'].iloc[0]
                sub_id = gin[gin['name'] == name_id]['study'].iloc[0]
                if type_id == type and sub_id == sub:
                    count = count + 1
                    mean = mean + nib.load(jac).get_fdata()     
            print(count)
            mean = mean / count
            out_name = "mean_jac_" + str(type) + "_" + str(sub) + ".nii.gz"
            target = os.path.join("output", "GIN", folder, out_name)
            image = nib.Nifti1Image(mean, affine, header)
            nib.save(image, target)
            print(target)
            print('-'*10)
            
for i, folder in enumerate(folders):
    for sub in subs:
        jacs = sorted(glob(os.path.join("output", "GIN", folder, "Log10Jacobian_Deformable", "*.nii.gz")))
        for type in types:
            count = 0
            mean_jac = nib.load(os.path.join("output", "GIN", folder, "mean_jac_" + str(type) + "_" + str(sub) + ".nii.gz")).get_fdata()
            std = np.zeros(std_jac_nib.shape)
            for jac in jacs:
                name_id = jac.split('/')[-1].split('_')[1]
                if 'kfold' in folder:
                    name_kfold = jac.split('/')[-1].split('_')[2] + '_' + jac.split('/')[-1].split('_')[3]
                    name_id = kfold[kfold['fold_name'] == name_kfold]['name'].iloc[0]
                type_id = gin[gin['name'] == name_id]['type'].iloc[0]                
                sub_id = gin[gin['name'] == name_id]['study'].iloc[0]
                if type_id == type and sub_id == sub:
                    count = count + 1
                    std = std + (nib.load(jac).get_fdata())**2   
            print(count)
            std = (std / count - mean_jac**2)**(1/2)
            image = nib.Nifti1Image(std, affine, header)
            out_name = "std_jac_" + str(type) + "_" + str(sub) + ".nii.gz"
            target = os.path.join("output", "GIN", folder, out_name)
            nib.save(image, target)
            print(target)
            print('-'*10)
            
for i, folder in enumerate(folders):
    std_jac_global = nib.load(os.path.join("output", "GIN", "ith_lambda1_3", "std_jac.nii.gz"))
    affine = std_jac_global.affine
    header = std_jac_global.header
    std_jac_global = std_jac_global.get_fdata() 
    for sub in subs:       
        mean_jac_wt = nib.load(os.path.join("output", "GIN", folder, "mean_jac_WT_" + sub + ".nii.gz")).get_fdata()
        mean_jac_ko = nib.load(os.path.join("output", "GIN", folder, "mean_jac_KO_" + sub + ".nii.gz")).get_fdata()        
        std_jac_wt = nib.load(os.path.join("output", "GIN", folder, "std_jac_WT_" + sub + ".nii.gz")).get_fdata()
        std_jac_ko = nib.load(os.path.join("output", "GIN", folder, "std_jac_KO_" + sub + ".nii.gz")).get_fdata()
                
        if sub == 'MAP6':
            n_ko = 5
            n_wt = 6
        else:
            n_ko = 8
            n_wt = 8    
        
        sp = (((n_wt-1)*std_jac_wt**2 + (n_ko-1)*std_jac_ko**2)/(n_wt+n_ko-2))**(1/2)
        denom = sp * (1/n_wt + 1/n_ko)**(1/2)
        t_test = (mean_jac_ko - mean_jac_wt) / denom
        image = nib.Nifti1Image(t_test, affine, header)        
        target = os.path.join("output", "GIN", folder, "ttest_" + sub + "_" + studies[i] + ".nii.gz")
        nib.save(image, target)       

8
output/GIN/ANTS/mean_jac_WT_SVBP.nii.gz
----------
8
output/GIN/ANTS/mean_jac_KO_SVBP.nii.gz
----------
6
output/GIN/ANTS/mean_jac_WT_MAP6.nii.gz
----------
5
output/GIN/ANTS/mean_jac_KO_MAP6.nii.gz
----------
8
output/GIN/ANTS-Mean/mean_jac_WT_SVBP.nii.gz
----------
8
output/GIN/ANTS-Mean/mean_jac_KO_SVBP.nii.gz
----------
6
output/GIN/ANTS-Mean/mean_jac_WT_MAP6.nii.gz
----------
5
output/GIN/ANTS-Mean/mean_jac_KO_MAP6.nii.gz
----------
8
output/GIN/ANTS-Mask/mean_jac_WT_SVBP.nii.gz
----------
8
output/GIN/ANTS-Mask/mean_jac_KO_SVBP.nii.gz
----------
6
output/GIN/ANTS-Mask/mean_jac_WT_MAP6.nii.gz
----------
5
output/GIN/ANTS-Mask/mean_jac_KO_MAP6.nii.gz
----------
8
output/GIN/dl-iris/mean_jac_WT_SVBP.nii.gz
----------
8
output/GIN/dl-iris/mean_jac_KO_SVBP.nii.gz
----------
6
output/GIN/dl-iris/mean_jac_WT_MAP6.nii.gz
----------
5
output/GIN/dl-iris/mean_jac_KO_MAP6.nii.gz
----------
8
output/GIN/Gin_kfold/mean_jac_WT_SVBP.nii.gz
----------
8
output/GIN/Gin_kfold/mean_jac_KO_SVBP.ni

/tmp/ipykernel_704155/222379023.py:86: RuntimeWarning: invalid value encountered in sqrt
  std = (std / count - mean_jac**2)**(1/2)


output/GIN/ANTS/std_jac_WT_SVBP.nii.gz
----------
8
output/GIN/ANTS/std_jac_KO_SVBP.nii.gz
----------
6
output/GIN/ANTS/std_jac_WT_MAP6.nii.gz
----------
5
output/GIN/ANTS/std_jac_KO_MAP6.nii.gz
----------
8
output/GIN/ANTS-Mean/std_jac_WT_SVBP.nii.gz
----------
8
output/GIN/ANTS-Mean/std_jac_KO_SVBP.nii.gz
----------
6
output/GIN/ANTS-Mean/std_jac_WT_MAP6.nii.gz
----------
5
output/GIN/ANTS-Mean/std_jac_KO_MAP6.nii.gz
----------
8
output/GIN/ANTS-Mask/std_jac_WT_SVBP.nii.gz
----------
8
output/GIN/ANTS-Mask/std_jac_KO_SVBP.nii.gz
----------
6
output/GIN/ANTS-Mask/std_jac_WT_MAP6.nii.gz
----------
5
output/GIN/ANTS-Mask/std_jac_KO_MAP6.nii.gz
----------
8
output/GIN/dl-iris/std_jac_WT_SVBP.nii.gz
----------
8
output/GIN/dl-iris/std_jac_KO_SVBP.nii.gz
----------
6
output/GIN/dl-iris/std_jac_WT_MAP6.nii.gz
----------
5
output/GIN/dl-iris/std_jac_KO_MAP6.nii.gz
----------
8
output/GIN/Gin_kfold/std_jac_WT_SVBP.nii.gz
----------
8
output/GIN/Gin_kfold/std_jac_KO_SVBP.nii.gz
----------
6
ou

/tmp/ipykernel_704155/222379023.py:114: RuntimeWarning: invalid value encountered in true_divide
  t_test = (mean_jac_ko - mean_jac_wt) / denom


In [20]:
folders = ["pairwise_zero_nosym_mean", 
           "pairwise_zero_nosym",
           "ith_best_aug_nosym",
           "ith_lambda1_3",
           "lambda1_1.0",
           "Gin_kfold",
           "dl-iris",
          ]
names = ["Pair_zero_mean",
        "Pair_zero",
        "Pair_aug",
        "Pair_noaug",
        "OP",
        "DL-gin-3fold",
        "DL-iris",
        ]

for i, folder in enumerate(folders):
    mris = os.path.join("output", "GIN", folder, "MRI_N4_Registration_Deformable", "*_Reg.nii.gz")
    labels = os.path.join("output", "GIN", folder, "Labels", "*_labels.nii.gz")
    cmd = "fslmerge -t isbi_serie_" + names[i] + ".nii.gz " + mris
    print(cmd)
    cmd = "fslmaths isbi_serie_" + names[i] + ".nii.gz -Tmean isbi_mean_" + names[i] + ".nii.gz"
    print(cmd)
    cmd = "fslmaths isbi_serie_" + names[i] + ".nii.gz -Tstd isbi_std_" + names[i] + ".nii.gz"
    print(cmd)
    cmd = "ImageMath 3 isbi_labels_" + names[i] + ".nii.gz MajorityVoting " + labels
    print(cmd)    

fslmerge -t isbi_serie_Pair_zero_mean.nii.gz output/GIN/pairwise_zero_nosym_mean/MRI_N4_Registration_Deformable/*_Reg.nii.gz
fslmaths isbi_serie_Pair_zero_mean.nii.gz -Tmean isbi_mean_Pair_zero_mean.nii.gz
fslmaths isbi_serie_Pair_zero_mean.nii.gz -Tstd isbi_std_Pair_zero_mean.nii.gz
ImageMath 3 isbi_labels_Pair_zero_mean.nii.gz MajorityVoting output/GIN/pairwise_zero_nosym_mean/Labels/*_labels.nii.gz
fslmerge -t isbi_serie_Pair_zero.nii.gz output/GIN/pairwise_zero_nosym/MRI_N4_Registration_Deformable/*_Reg.nii.gz
fslmaths isbi_serie_Pair_zero.nii.gz -Tmean isbi_mean_Pair_zero.nii.gz
fslmaths isbi_serie_Pair_zero.nii.gz -Tstd isbi_std_Pair_zero.nii.gz
ImageMath 3 isbi_labels_Pair_zero.nii.gz MajorityVoting output/GIN/pairwise_zero_nosym/Labels/*_labels.nii.gz
fslmerge -t isbi_serie_Pair_aug.nii.gz output/GIN/ith_best_aug_nosym/MRI_N4_Registration_Deformable/*_Reg.nii.gz
fslmaths isbi_serie_Pair_aug.nii.gz -Tmean isbi_mean_Pair_aug.nii.gz
fslmaths isbi_serie_Pair_aug.nii.gz -Tstd isbi_s

In [21]:
folders = ["GIN", 
           "GIN_Mean",
          ]
names = ["SyN",
        "SyN-Mean",
        ]

for i, folder in enumerate(folders):
    mris = os.path.join("dataset3", folder, "MRI", "*_affine_deformable.nii.gz")
    labels = os.path.join("dataset3", folder, "Labels", "*_deformable_convert.nii.gz")
    cmd = "fslmerge -t isbi_serie_" + names[i] + ".nii.gz " + mris
    print(cmd)
    cmd = "fslmaths isbi_serie_" + names[i] + ".nii.gz -Tmean isbi_mean_" + names[i] + ".nii.gz"
    print(cmd)
    cmd = "fslmaths isbi_serie_" + names[i] + ".nii.gz -Tstd isbi_std_" + names[i] + ".nii.gz"
    print(cmd)
    cmd = "ImageMath 3 isbi_labels_" + names[i] + ".nii.gz MajorityVoting " + labels
    print(cmd)    

fslmerge -t isbi_serie_SyN.nii.gz dataset3/GIN/MRI/*_affine_deformable.nii.gz
fslmaths isbi_serie_SyN.nii.gz -Tmean isbi_mean_SyN.nii.gz
fslmaths isbi_serie_SyN.nii.gz -Tstd isbi_std_SyN.nii.gz
ImageMath 3 isbi_labels_SyN.nii.gz MajorityVoting dataset3/GIN/Labels/*_deformable_convert.nii.gz
fslmerge -t isbi_serie_SyN-Mean.nii.gz dataset3/GIN_Mean/MRI/*_affine_deformable.nii.gz
fslmaths isbi_serie_SyN-Mean.nii.gz -Tmean isbi_mean_SyN-Mean.nii.gz
fslmaths isbi_serie_SyN-Mean.nii.gz -Tstd isbi_std_SyN-Mean.nii.gz
ImageMath 3 isbi_labels_SyN-Mean.nii.gz MajorityVoting dataset3/GIN_Mean/Labels/*_deformable_convert.nii.gz


In [3]:
folders = ["GIN", 
          ]
names = ["Affine",
        ]

for i, folder in enumerate(folders):
    mris = os.path.join("dataset3", folder, "MRI", "*_affine.nii.gz")
    labels = os.path.join("dataset3", folder, "Labels", "*_affine_convert.nii.gz")
    cmd = "fslmerge -t isbi_serie_" + names[i] + ".nii.gz " + mris
    print(cmd)
    cmd = "fslmaths isbi_serie_" + names[i] + ".nii.gz -Tmean isbi_mean_" + names[i] + ".nii.gz"
    print(cmd)
    cmd = "fslmaths isbi_serie_" + names[i] + ".nii.gz -Tstd isbi_std_" + names[i] + ".nii.gz"
    print(cmd)
    cmd = "ImageMath 3 isbi_labels_" + names[i] + ".nii.gz MajorityVoting " + labels
    print(cmd)    

fslmerge -t isbi_serie_Affine.nii.gz dataset3/GIN/MRI/*_affine.nii.gz
fslmaths isbi_serie_Affine.nii.gz -Tmean isbi_mean_Affine.nii.gz
fslmaths isbi_serie_Affine.nii.gz -Tstd isbi_std_Affine.nii.gz
ImageMath 3 isbi_labels_Affine.nii.gz MajorityVoting dataset3/GIN/Labels/*_affine_convert.nii.gz


In [4]:
folders = ["GIN_Mask", 
          ]
names = ["SyN_Mask",
        ]

for i, folder in enumerate(folders):
    mris = os.path.join("dataset3", folder, "MRI", "*_affine_deformable.nii.gz")
    labels = os.path.join("dataset3", folder, "Labels", "*_affine_deformable.nii.gz")
    cmd = "fslmerge -t isbi_serie_" + names[i] + ".nii.gz " + mris
    print(cmd)
    cmd = "fslmaths isbi_serie_" + names[i] + ".nii.gz -Tmean isbi_mean_" + names[i] + ".nii.gz"
    print(cmd)
    cmd = "fslmaths isbi_serie_" + names[i] + ".nii.gz -Tstd isbi_std_" + names[i] + ".nii.gz"
    print(cmd)
    cmd = "ImageMath 3 isbi_labels_" + names[i] + ".nii.gz MajorityVoting " + labels
    print(cmd)    

fslmerge -t isbi_serie_SyN_Mask.nii.gz dataset3/GIN_Mask/MRI/*_affine_deformable.nii.gz
fslmaths isbi_serie_SyN_Mask.nii.gz -Tmean isbi_mean_SyN_Mask.nii.gz
fslmaths isbi_serie_SyN_Mask.nii.gz -Tstd isbi_std_SyN_Mask.nii.gz
ImageMath 3 isbi_labels_SyN_Mask.nii.gz MajorityVoting dataset3/GIN_Mask/Labels/*_affine_deformable.nii.gz


In [54]:
from glob import glob

names = ["Pair_zero_mean",
        "Pair_zero",
        "Pair_aug",
        "Pair_noaug",
        "OP",
        "DL-gin-3fold",
        "DL-iris",
        "SyN",
        "SyN-Mean",
        "SyN-Mask",         
        ]
folders = ["pairwise_zero_nosym_mean", 
           "pairwise_zero_nosym",
           "ith_best_aug_nosym",
           "ith_lambda1_3",
           "lambda1_1.0",
           "Gin_kfold",
           "dl-iris",
           "ANTS", 
           "ANTS-Mean",
           "ANTS-Mask",
          ]

for i in range(len(folders)):
    if len(folders) == len(names):
        dl = not "syn" in names[i].lower()
        if dl:
            warps = sorted(glob(os.path.join("output", "GIN", folders[i], "DeformableWarp", "*_warp.nii.gz")))
        else:
            warps = sorted(glob(os.path.join("output", "GIN", folders[i], "DeformableWarp", "*_deformablewarp.nii.gz")))
            if len(warps) == 0:
                warps = sorted(glob(os.path.join("output", "GIN", folders[i], "DeformableWarp", "*_warp.nii.gz")))
        log10jacs = sorted(glob(os.path.join("output", "GIN", folders[i], "Log10Jacobian_Deformable", "Log10JacobianDeformable*")))
        log10jacs_cmd = os.path.join("output", "GIN", folders[i], "Log10Jacobian_Deformable", "Log10JacobianDeformable*")
        if len(warps) != len(log10jacs) and len(warps) == 27:  
            outdir = os.path.join('output', 'GIN', folders[i], 'Log10Jacobian_Deformable')
            try:
                os.mkdir(outdir)
            except:
                pass
            for j in range(len(warps)):            
                name = warps[j].split('/')[-1].split('.')[0]
                outname = os.path.join(outdir, "Log10JacobianDeformable_" + name + ".nii.gz")
                jacobian_command = "CreateJacobianDeterminantImage 3 " + str(warps[j]) + " " + str(outname) + " 1 1"
                subprocess.call(jacobian_command.split(" "))
        elif len(warps) == 0:
            print(folders[i])
        if len(warps) == len(log10jacs) == 27:
            cmd = "fslmerge -t isbi_jac_serie_" + names[i] + ".nii.gz " + log10jacs_cmd
            print(cmd)
            cmd = "fslmaths isbi_jac_serie_" + names[i] + ".nii.gz -Tmean isbi_jac_mean_" + names[i] + ".nii.gz"
            print(cmd)
            cmd = "fslmaths isbi_jac_serie_" + names[i] + ".nii.gz -Tstd isbi_jac_std_" + names[i] + ".nii.gz"
            print(cmd)

fslmerge -t isbi_jac_serie_Pair_zero_mean.nii.gz output/GIN/pairwise_zero_nosym_mean/Log10Jacobian_Deformable/Log10JacobianDeformable*
fslmaths isbi_jac_serie_Pair_zero_mean.nii.gz -Tmean isbi_jac_mean_Pair_zero_mean.nii.gz
fslmaths isbi_jac_serie_Pair_zero_mean.nii.gz -Tstd isbi_jac_std_Pair_zero_mean.nii.gz
fslmerge -t isbi_jac_serie_Pair_zero.nii.gz output/GIN/pairwise_zero_nosym/Log10Jacobian_Deformable/Log10JacobianDeformable*
fslmaths isbi_jac_serie_Pair_zero.nii.gz -Tmean isbi_jac_mean_Pair_zero.nii.gz
fslmaths isbi_jac_serie_Pair_zero.nii.gz -Tstd isbi_jac_std_Pair_zero.nii.gz
fslmerge -t isbi_jac_serie_Pair_aug.nii.gz output/GIN/ith_best_aug_nosym/Log10Jacobian_Deformable/Log10JacobianDeformable*
fslmaths isbi_jac_serie_Pair_aug.nii.gz -Tmean isbi_jac_mean_Pair_aug.nii.gz
fslmaths isbi_jac_serie_Pair_aug.nii.gz -Tstd isbi_jac_std_Pair_aug.nii.gz
fslmerge -t isbi_jac_serie_Pair_noaug.nii.gz output/GIN/ith_lambda1_3/Log10Jacobian_Deformable/Log10JacobianDeformable*
fslmaths isbi

In [7]:
from glob import glob
import os

names = ["Pair_zero_new",   
         "Pair_null",
        ]
folders = ["pairwise_zero_nosym_fix", 
           "ith_null",
          ]

for i in range(len(folders)):
    if len(folders) == len(names):
        dl = not "syn" in names[i].lower()
        if dl:
            warps = sorted(glob(os.path.join("output", "GIN", folders[i], "DeformableWarp", "*_warp.nii.gz")))
        else:
            warps = sorted(glob(os.path.join("output", "GIN", folders[i], "DeformableWarp", "*_deformablewarp.nii.gz")))
            if len(warps) == 0:
                warps = sorted(glob(os.path.join("output", "GIN", folders[i], "DeformableWarp", "*_warp.nii.gz")))
        log10jacs = sorted(glob(os.path.join("output", "GIN", folders[i], "Log10Jacobian_Deformable", "Log10JacobianDeformable*")))
        log10jacs_cmd = os.path.join("output", "GIN", folders[i], "Log10Jacobian_Deformable", "Log10JacobianDeformable*")
        if len(warps) != len(log10jacs) and len(warps) == 27:  
            outdir = os.path.join('output', 'GIN', folders[i], 'Log10Jacobian_Deformable')
            try:
                os.mkdir(outdir)
            except:
                pass
            for j in range(len(warps)):            
                name = warps[j].split('/')[-1].split('.')[0]
                outname = os.path.join(outdir, "Log10JacobianDeformable_" + name + ".nii.gz")
                jacobian_command = "CreateJacobianDeterminantImage 3 " + str(warps[j]) + " " + str(outname) + " 1 1"
                subprocess.call(jacobian_command.split(" "))
        elif len(warps) == 0:
            print(folders[i])
        if len(warps) == len(log10jacs) == 27:
            cmd = "fslmerge -t isbi_jac_serie_" + names[i] + ".nii.gz " + log10jacs_cmd
            print(cmd)
            cmd = "fslmaths isbi_jac_serie_" + names[i] + ".nii.gz -Tmean isbi_jac_mean_" + names[i] + ".nii.gz"
            print(cmd)
            cmd = "fslmaths isbi_jac_serie_" + names[i] + ".nii.gz -Tstd isbi_jac_std_" + names[i] + ".nii.gz"
            print(cmd)

fslmerge -t isbi_jac_serie_Pair_zero_new.nii.gz output/GIN/pairwise_zero_nosym_fix/Log10Jacobian_Deformable/Log10JacobianDeformable*
fslmaths isbi_jac_serie_Pair_zero_new.nii.gz -Tmean isbi_jac_mean_Pair_zero_new.nii.gz
fslmaths isbi_jac_serie_Pair_zero_new.nii.gz -Tstd isbi_jac_std_Pair_zero_new.nii.gz
fslmerge -t isbi_jac_serie_Pair_null.nii.gz output/GIN/ith_null/Log10Jacobian_Deformable/Log10JacobianDeformable*
fslmaths isbi_jac_serie_Pair_null.nii.gz -Tmean isbi_jac_mean_Pair_null.nii.gz
fslmaths isbi_jac_serie_Pair_null.nii.gz -Tstd isbi_jac_std_Pair_null.nii.gz


In [8]:
import os
from glob import glob

names = ["Pair_zero_new",   
         "Pair_null",
        ]
folders = ["pairwise_zero_nosym_fix", 
           "ith_null",
          ]

for i, folder in enumerate(folders):
    mris = os.path.join("output", "GIN", folder, "MRI_N4_Registration_Deformable", "*_Reg.nii.gz")
    labels = os.path.join("output", "GIN", folder, "Labels", "*_labels.nii.gz")
    cmd = "fslmerge -t isbi_serie_" + names[i] + ".nii.gz " + mris
    print(cmd)
    cmd = "fslmaths isbi_serie_" + names[i] + ".nii.gz -Tmean isbi_mean_" + names[i] + ".nii.gz"
    print(cmd)
    cmd = "fslmaths isbi_serie_" + names[i] + ".nii.gz -Tstd isbi_std_" + names[i] + ".nii.gz"
    print(cmd)
    cmd = "ImageMath 3 isbi_labels_" + names[i] + ".nii.gz MajorityVoting " + labels
    print(cmd)    

fslmerge -t isbi_serie_Pair_zero_new.nii.gz output/GIN/pairwise_zero_nosym_fix/MRI_N4_Registration_Deformable/*_Reg.nii.gz
fslmaths isbi_serie_Pair_zero_new.nii.gz -Tmean isbi_mean_Pair_zero_new.nii.gz
fslmaths isbi_serie_Pair_zero_new.nii.gz -Tstd isbi_std_Pair_zero_new.nii.gz
ImageMath 3 isbi_labels_Pair_zero_new.nii.gz MajorityVoting output/GIN/pairwise_zero_nosym_fix/Labels/*_labels.nii.gz
fslmerge -t isbi_serie_Pair_null.nii.gz output/GIN/ith_null/MRI_N4_Registration_Deformable/*_Reg.nii.gz
fslmaths isbi_serie_Pair_null.nii.gz -Tmean isbi_mean_Pair_null.nii.gz
fslmaths isbi_serie_Pair_null.nii.gz -Tstd isbi_std_Pair_null.nii.gz
ImageMath 3 isbi_labels_Pair_null.nii.gz MajorityVoting output/GIN/ith_null/Labels/*_labels.nii.gz
